# Alumno: Gerardo de Miguel González

# Machine Learning II (Master en Data Science, UIMP-UC) 

# Tarea: Redes Bayesianas

### Fecha de entrega: 7 Abril 2019

Responde a las preguntas planteadas y aporta el código en R utilizado para responderlas. Se debe enviar un único documento con texto y código entrelazado, bien sea en formato pdf o html.

## Introducción a la práctica

Utilizaremos un conjunto de datos diarios de ocurrencia de meteoros en el aeropuerto de Parayas (Santander), considerando una serie de variables meteorológicas registradas diariamente durante un periodo aproximado de 10 años (3286 registros), en concreto: precipitación, nieve, granizo, tormenta, niebla, rocío, escarcha, nieve en el suelo, neblina y racha máxima de viento superior a 50 km/h. Los datos se encuentran guardados en el fichero de texto `meteoro.txt`. Las variables se encuentran discretizadas de forma binaria (ocurrencia/ausencia).

En este caso, y a través del asesoramiento de un meteorólogo, se ha elaborado el siguiente diagrama acíclico dirigido (DAG), que recoge las relaciones de dependencia entre las variables:

![modelo experto](dag-meteo-modelo-experto.png)

## Ejercicio 1 (2.5 Puntos)

In [1]:
suppressMessages(library('bnlearn'))

In [2]:
meteoro <- read.table('meteoro.txt', header = TRUE)

In [3]:
str(meteoro)

'data.frame':	3286 obs. of  10 variables:
 $ lluvia    : Factor w/ 2 levels "n","s": 2 2 2 2 2 2 2 2 2 1 ...
 $ nieve     : Factor w/ 2 levels "n","s": 1 1 1 1 1 1 1 1 1 1 ...
 $ granizo   : Factor w/ 2 levels "n","s": 1 1 1 1 2 2 1 1 1 1 ...
 $ tormenta  : Factor w/ 2 levels "n","s": 1 1 2 1 2 2 2 1 1 1 ...
 $ niebla    : Factor w/ 2 levels "n","s": 1 1 1 1 1 1 1 1 1 1 ...
 $ rocio     : Factor w/ 2 levels "n","s": 1 1 1 1 1 1 1 1 1 2 ...
 $ escarcha  : Factor w/ 2 levels "n","s": 1 1 1 1 1 1 1 1 1 1 ...
 $ nieveSuelo: Factor w/ 2 levels "n","s": 1 1 1 1 1 1 1 1 1 1 ...
 $ neblina   : Factor w/ 2 levels "n","s": 1 1 1 1 1 1 1 1 2 2 ...
 $ viento    : Factor w/ 2 levels "n","s": 2 2 2 2 2 2 2 2 1 1 ...


In [4]:
head(meteoro)

lluvia,nieve,granizo,tormenta,niebla,rocio,escarcha,nieveSuelo,neblina,viento
s,n,n,n,n,n,n,n,n,s
s,n,n,n,n,n,n,n,n,s
s,n,n,s,n,n,n,n,n,s
s,n,n,n,n,n,n,n,n,s
s,n,s,s,n,n,n,n,n,s
s,n,s,s,n,n,n,n,n,s


In [6]:
nodos <- colnames(meteoro)
print(nodos)

 [1] "lluvia"     "nieve"      "granizo"    "tormenta"   "niebla"    
 [6] "rocio"      "escarcha"   "nieveSuelo" "neblina"    "viento"    


* A partir del grafo dado por el experto, determina la expresión que refleja la factorización de la distribución global, y escríbela como una secuencia de caracteres del tipo generado por la función `modelstring` del paquete de R `bnlearn`. 

In [38]:
# https://stackoverflow.com/questions/6329962/split-code-over-multiple-lines-in-an-r-script
meteoro_pdag <- paste('[viento][lluvia|viento][rocio|viento][tormenta|lluvia][niebla|rocio:viento]',
                      '[escarcha|rocio][nieve|tormenta][granizo|tormenta][neblina|niebla][nieveSuelo|nieve]',
                      sep="")
print(meteoro_pdag)

[1] "[viento][lluvia|viento][rocio|viento][tormenta|lluvia][niebla|rocio:viento][escarcha|rocio][nieve|tormenta][granizo|tormenta][neblina|niebla][nieveSuelo|nieve]"


NOTA: Como ya se comentado anteriormente, es importante evitar tildes a la hora de nombrar los nodos. Tampoco son recomendables espacios en blanco ni ningún tipo de carácter especial. 

* A continuación, introduce el DAG en R utilizando la definicíon del modelo que acabas de crear

In [41]:
# http://www.bnlearn.com/documentation/man/modelstring.html
pdag <- bnlearn::model2network(string = meteoro_pdag)

In [42]:
pdag2 <- bnlearn::as.bn(meteoro_pdag)

In [45]:
all.equal(target = pdag, current = pdag2)

[1] TRUE

In [46]:
class(pdag)

[1] "bn"

* Obten una lista de los padres e hijos de cada uno de los nodos

In [54]:
print(bnlearn::nodes(object = pdag))

 [1] "escarcha"   "granizo"    "lluvia"     "neblina"    "niebla"    
 [6] "nieve"      "nieveSuelo" "rocio"      "tormenta"   "viento"    


In [53]:
bnlearn::parents(x = pdag, node = )

ERROR: Error in check.nodes(nodes = node, graph = x, max.nodes = 1): at most 1 node(s) needed.


* Realiza una lista de todas las conexiones fundamentales presentes en el grafo, y realiza una clasificación de cada una de ellas determinando si es una estructura en serie, divergente o convergente.

* ¿Se observa alguna *v-estructura* en el grafo?

* Intenta introducir un arco que parta del nodo neblina y se dirija hacia escarcha, y otro que vaya de granizo a lluvia. Comenta qué sucede en cada caso, y si el resultado daría lugar a una red bayesiana válida.

* Determina la manta de Markov del nodo `rocio`

* Introduce un arco que parta del nodo Lluvia y se dirija al nodo Niebla, ¿cambia la manta de Markov del nodo `rocio`? En caso afirmativo, ¿cómo lo hace?

NOTA: Para continuar, elimina el arco definido anteriormente entre los nodos Lluvia y Niebla, recuperando el DAG original.

## Ejercicio 2 (2.5 Puntos)

* Considerando la información proporcionada por el conjunto de datos `meteor`, construye la red bayesiana utilizando el método bayesiano de estimación paramétrica.

* ¿Cuál sería el número potencial de parámetros (sin usar la red bayesiana) del modelo para calcular la probabilidad global si no utilizasemos el DAG?

* ¿Cuántos parámetros tiene la distribución global dada por la red bayesiana?

* Determina el número de parámetros asociado a cada una de las distribuciones locales

* Obtén las tablas de probabilidad condicional asociadas los nodos `granizo` y `niebla`. Ahora representa la información de cada tabla en sendos gráficos.

## Ejercicio 3 (2.5 Puntos)

Una vez construída la red hemos establecido la base de conocimiento del sistema inteligente. A continuación se puede calcular la probabilidad de cualquier variable o conjunto de variables condicionadas a cualquier evidencia que se tenga disponible para un problema dado, es decir, realizar la inferencia. 

Observa la estructura del DAG y repasa la teoría y el concepto de _d-separación_. Responde razonadamente si las siguientes afirmaciones son verdaderas o falsas, utilizando únicamente la estructura del DAG y el concepto de d-separación:

+ La nieve y el granizo son fenómenos independientes a priori
+ La nieve y el granizo son fenómenos independientes dado que haya habido tormenta
+ La nieve en el suelo y la neblina son fenómenos independientes
+ La nieve en el suelo y la neblina son fenómenos independientes dado que haya habido tormenta

#### Inferencia exacta

NOTA: En este apartado se deberá aplicar la inferencia exacta. 

Conocido que en un día dado se han producido tormentas, calcula cómo afecta este hecho a la probabilidad de que se produzcan los siguientes fenómenos meteorológicos:

* Que llueva, $P(lluvia = s | tormenta = s)$
* Que haya rachas de viento superiores a 50 Km/h, $P(viento = s | tormenta = s)$
* Que llueva y que además las rachas de viento superen los 50 Km/h, $P(lluvia = s, viento = n | tormenta = s)$

A partir de la información revelada por la red bayesiana, sabiendo que un día se producen tormentas:       

* ¿Hay mayor probabilidad de que llueva cuando se producen tormentas que cualquier otro día?
* ¿Aumenta o disminuye la probabilidad de tener rachas de viento mayores de 50 Km/h cuando se produce tormenta? ¿Cuánto?

#### Inferencia aproximada

Dado el ejercicio anterior, repetir ahora el ejercicio mediante inferencia aproximada, calculando para cada una de las estimaciones 100 realizaciones. Para cada una de las respuestas anteriores, representa un diagrama de cajas que represente la dispersión en la estimación de la probabilidad, marcando además el valor obtenido de manera exacta en el apartado anterior.

## Ejercicio 4 (2.5 Puntos)

Como hemos visto, es posible realizar un aprendizaje automático de la estructura del DAG a partir de los datos, usando algoritmos específicos para ello. Durante las clases hemos visto el ejemplo del algoritmo _hill-climbing_, aunque como vimos, hay otras posibilidades. Tambien hemos visto que podemos combinar nuestra experiencia y el aprendizaje automático definiendo previamente relaciones entre variables que queremos introducir o descartar en el DAG resultante. Además, se ha explicado que existen _scores_ que sirven como criterio para evaluar la fuerza de la dependendencia entre nodos de la red y comparar la bondad de ajuste del modelo.

* Evalúa la significación de los arcos dibujados por el experto en el actual DAG utilizando el estadístico $\chi^2$. ¿Hay algún arco no significativo? ¿Cuáles son los tres pares de nodos que presentan un arco de unión más fuerte?

Además del algoritmo _hill-climbing_, existe otro algoritmo popular de tipo "voraz" denominado _Tabu search_. En `bnlearn` se encuentra implementado a traves de la función `tabu`, y los argumentos de entrada son similares a los vistos para _hill-climbing_. 

* Compara el score global (BIC) obtenido por el DAG inicial, con los obtenidos por los DAG aprendidos de forma automática con los algoritmos _tabu_ y _hill-climbing_. ¿Cuál obtiene mejor score?

   * Ahora, para comparar el DAG original con los dos nuevos DAG tabu y hill-climbing, dibuja los tres utilizando la función `graphviz.plot`. A la luz de los grafos obtenidos en cada caso, ¿cuál te parece que recoge mejor las relaciones causa-efecto entre variables?. Hay que tener en cuenta que los arcos de un grafo __no expresan causalidad__, sino simplemente dependencia entre variables en términos de probabilidad. 

* Ahora vuelve a aprender de forma automática el DAG usando tabu y hill-climbing, pero imponiendo las siguientes restricciones:
     1. Los arcos `viento --> lluvia`, `tormenta --> granizo` y `nieve --> nieveSuelo` deben quedar reflejados en el DAG.
     2. Ningún arco debe unir directamente la neblina con el granizo ni la niebla con la tormenta.

   * Vuelve a dibujar los DAG resultantes, y a partir del BIC obtenido por cada modelo, determina cuál es el mejor de todos poniéndolos en una tabla.

   * Computa la fuerza de la relación entre nodos de los dos últimos modelos que combinan nuestra experiencia con el aprendizaje automático. 

   * Comenta brevemente los resultados obtenidos tras combinar nuestro conocimiento con el aprendizaje automático.   